# ACTA 3. Preprocesado de los datos

## Índice
- [Introducción](#Introducción)
- [Cálculo de variables](#Cálulo-de-variables)
    - [Neighborhood](#Cálulo-de-variables#Neighborhood)
    - [Police_district](#Cálulo-de-variables#Police_district)
    - [Holiday](#Cálulo-de-variables#Holiday)
    - [Street](#Cálulo-de-variables#Street)
- [Ejecución Paralela del Procesado](#Ejecución-Paralela-del-Procesado)

## Introducción<a name="Introducción"></a>

El propósito de esta parte del proyecto es el de preparar nuestra base de datos para poder emplearla para la obtención de nuestro modelo de datos.
Para conseguir nuestro objetivo dividiremos esta etapa en dos, la primera consistirá en el cálculo de nuevas variables para lo que se empleará este archivo y  la segunda parte  comprenserá una limpieza de todos los datos obtenidos a partir de la unión de los incidentes limpios y el cálculo de estas nuevas variables. Posteriormente en la segunda parte también se aplicará la codificación de las variables categóricas para convertirlas en variables numéricas y de esta forma poder emplearlas en un futuro en caso de que apliquemos alguna téncica estadística que requiera datos numéricos.

Para comenzar con nuestra primera parte cargaremos nuestro conjunto de datos resultante de la limpieza de datos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from shapely.geometry import Point, shape
import geopandas as gpd
import urllib.request
import numpy as np
import holidays
from geopy.geocoders import Nominatim
import concurrent.futures
import time

In [ ]:
Incidents_c = pd.read_csv('Incidents_Clean_F.csv', delimiter = ',')
Incidents_c.columns

Index(['incident_datetime', 'incident_day_of_week', 'incident_code',
       'incident_category', 'incident_subcategory', 'resolution', 'latitude',
       'longitude'],
      dtype='object')

## Cálculo de variables<a name="Cálculo de variables"></a>

### Neighborhood <a name="Neighborhood"></a>

Para calcular la variable vecindario lo que haremos será crear una función que a partir del conjunto de datos de vecindarios de San Francisco en el que se defínen los polígonos de cada vecindario, localice a qué vecindario pertenece cada uno de nuestros registros a partir de la latitud y la longitud.
La descarga de este archivo se puede ver en el documento "ACTA 1. Obtención y extracción de los datos".

In [ ]:
#VECINDARIO
def assign_neighborhood(df, geojson_url):
    filename = 'neighborhoods.geojson'
    urllib.request.urlretrieve(geojson_url, filename)

    neighborhoods = gpd.read_file('neighborhoods.geojson')

    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    crs = {'init': 'epsg:4326'}
    incidents_geo = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

    result = gpd.sjoin(incidents_geo, neighborhoods, how="left", op='intersects')

    return result['neighborhoods_analysis_boundaries']



### Police_district <a name="Police_district"></a>

Calcularemos también el distrito policial al que pertenece cada uno de nuestros registros a partir del conjunto de datos Police Districs descargado en el documento "ACTA 1. Obtención y extracción de los datos".
Para ello crearemos una función y buscaremos cada registro en los distintos polígonos que define el archivo y en cuanto coincida devolverá el nombre del distrito.

In [ ]:

#DISTRITO POLICIAL

file_path = 'Current Police Districts.geojson'

with open(file_path) as f:
    js = json.load(f)
    districts = [{'name': feature['properties']['district'], 'geometry': shape(feature['geometry'])} for feature in js['features']]

def is_in_district(latitude, longitude):
    point = Point(longitude, latitude)
    for district in districts:
        if point.within(district['geometry']):
            return district['name']
    return None

### Holiday <a name="Holiday"></a>

También calcularemos la columna Holiday que nos determinará si el incidente analizada s eha producido en un día festivo en San Francisco o en California o en Ambos.
Para ello emplearemos la librería holidays y cargaremos los días festivos desde 2018 hasta 2023 y veremos si coincide con la fecha de nuestro incidente.

In [ ]:


#FESTIVO

ca_holidays = holidays.US(state='CA')
sf_holidays = holidays.US(state='CA', years=range(2018, 2024))


def is_holiday(date):
    if date in ca_holidays and date in sf_holidays:
        return "Both"
    elif date in ca_holidays:
        return "California"
    elif date in sf_holidays:
        return "San Francisco"
    else:
        return "None"

### Street <a name="Street"></a>

Por último, calcularemos la columna Street que nos determinará la calle en la que se ha producido el incidente. Para ello crearemos una función que empleará la librería Nominatim que devolverá la calle a partir de la latitud y la longitud.

In [ ]:


#CALLE
geolocator = Nominatim(user_agent='myGeocoder')

def get_street(longitude, latitude):
    try:
        location = geolocator.reverse((latitude, longitude), timeout=10)
        time.sleep(1)
        street = location.raw['address']['road'] if 'road' in location.raw['address'] else None
        print(f"Obtenido: {street}")
        return street
    except Exception as e:
        print(f"Error al obtener la dirección: {e}")
        return 'No calculado'




## Ejecución Paralela del Procesado<a name="Ejecución Paralela del Procesado"></a>

A continuación se ha definido código Python para ejecutar todas las funciones definidas anteriormente sobre nuestro conjunto de datos.
En una primera instancia se ejecutó en un único ordenar sin aplicar ninguna técnica de paralelización, al ver que tardaba más de 3 días se decidió aplicar técnicas de paralelización mediante la librería concurrent.features. Aún así, se vió que tardaba también más de 3 días y se dividión el dataset en dos conjuntos de datos que fueron procesados de forma paralela en dos ordenadores distintos y dividiento cada mitad en 8 partes, de esta forma se procesaron los las 16 particiones de forma paralela, 8 en cada ordenador.
Esto llevó un tiempo de 4 días y medio debido a la gran cantidad de datos que teníamos. El motivo por el que tardaba tanto e spor el cálculo de la calle que tenía que hacer una solicitud registro a registro para obtener la dirección completa y luego extraer la calle. Además que para asegurarnos de que funcionara, mostrábamos por pantalla el valor de la calle.

In [ ]:
def process_dataframe(df_part, filename):
    df_part['neighborhood'] = assign_neighborhood(df_part, 'neighborhoods.geojson')
    df_part['Police_district'] = df_part.apply(lambda row: is_in_district(row['latitude'], row['longitude']), axis=1)
    df_part['Holiday'] = pd.to_datetime(df_part['incident_datetime']).dt.date.apply(is_holiday)
    df_part['Street'] = df_part.apply(lambda row: get_street(row['longitude'], row['latitude']), axis=1)


    df_part.to_csv(filename, index=False)

    return df_part



n_parts = 8
parts = np.array_split(Incidents_c, n_parts)


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for i, part in enumerate(parts):
        filename = f'Incidents_part_{i}.csv'
        futures.append(executor.submit(process_dataframe, part, filename))


    dfs = []
    for future in concurrent.futures.as_completed(futures):
        df_part = future.result()
        dfs.append(df_part)


Incidents_sf = pd.concat(dfs, ignore_index=True)



c:\Python\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Python\lib\concurrent\futures\thread.py:58: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = self.fn(*self.args, **self.kwargs)
C:\Users\fatima.elaouni\AppData\Local\Temp\ipykernel_45548\2324694285.py:23: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd.sjoin(incidents_geo, neighborhoods, how="left", op='intersects')


Obtenido: 43rd Avenue


c:\Python\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Python\lib\concurrent\futures\thread.py:58: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = self.fn(*self.args, **self.kwargs)
C:\Users\fatima.elaouni\AppData\Local\Temp\ipykernel_45548\2324694285.py:23: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd.sjoin(incidents_geo, neighborhoods, how="left", op='intersects')


Obtenido: Fremont Street
Obtenido: Cesar Chavez Street
Obtenido: Buchanan Street
Obtenido: Cabrillo Street
Obtenido: Charles J. Brenham Place
Obtenido: Armstrong Avenue
Obtenido: San Jose Avenue
Obtenido: Benton Avenue
Obtenido: None
Obtenido: Tapia Drive
Obtenido: Noriega Street
Obtenido: Bartlett Street
Obtenido: Octavia Street
Obtenido: 24th Street
Obtenido: Hayes Street
Obtenido: Delancey Street
Obtenido: None
Obtenido: Haight Street
Obtenido: Powell Street
Obtenido: North Point Street
Obtenido: Rickard Street
Obtenido: Lombard Street
Obtenido: Geary Street
Obtenido: Orizaba Avenue
Obtenido: 17th Street
Obtenido: Sutter Street
Obtenido: Fillmore Street
Obtenido: Quesada Avenue
Obtenido: 23rd Street
Obtenido: 21st Street
Obtenido: Zoe Street
Obtenido: 19th Street
Obtenido: Eddy Street
Obtenido: Powell Street
Obtenido: 24th Street
Obtenido: Hayes Street
Obtenido: Silver Avenue
Obtenido: Cyril Magnin Street
Obtenido: Kirkham Street
Obtenido: Ortega Street
Obtenido: 4th Street
Obtenido